# Chain of Thought (CoT) Prompting Tutorial

## Overview

This tutorial delves into Chain of Thought (CoT) prompting, an advanced technique in prompt engineering that guides AI models to dissect complex problems into detailed, step-by-step reasoning processes. We'll explore the implementation of CoT prompting using Amazon Nova via OpenRouter and LangChain.

## Motivation

With the rapid advancement of AI language models, there's a growing demand for outputs that are more transparent, logical, and verifiable. CoT prompting addresses this need by encouraging models to articulate their thought process, similar to how humans tackle complex problems. This approach not only enhances the accuracy of AI-generated responses but also increases their interpretability and reliability.

## Key Components

1. **CoT Prompting Fundamentals**: Introduction to the core concepts and basic implementation.
2. **Advanced CoT Strategies**: Exploration of more sophisticated CoT methodologies.
3. **Comparative Study**: Analysis of the distinctions between traditional and CoT prompting techniques.
4. **Practical Applications**: Utilizing CoT for a variety of complex problem-solving scenarios.

## Setup

Let's start by importing the necessary libraries and setting up our environment.

In [1]:
from os import getenv

from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

load_dotenv()

# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="amazon/nova-lite-v1",
)

## Basic Chain of Thought Prompting

Let's start with a simple example to demonstrate the difference between a standard prompt and a Chain of Thought prompt.

In [2]:
# Standard prompt
standard_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question conciesly: {question}.",
)

# Chain of Thought prompt
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question step by step conciesly: {question}",
)

# Create chains
standard_chain = standard_prompt | llm
cot_chain = cot_prompt | llm

# Example question
question = "If a train travels 120 km in 2 hours, what is its average speed in km/h?"

# Get responses
standard_response = standard_chain.invoke(question).content
cot_response = cot_chain.invoke(question).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)

Standard Response:
60 km/h

Chain of Thought Response:
To find the average speed, divide the total distance by the total time.

1. Distance = 120 km
2. Time = 2 hours

Average speed = Distance / Time = 120 km / 2 hours = 60 km/h

The average speed is 60 km/h.


## Advanced Chain of Thought Techniques

Now, let's explore a more advanced CoT technique that encourages multi-step reasoning.

In [3]:
advanced_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""Solve the following problem step by step. For each step:
1. State what you're going to calculate
2. Write the formula you'll use (if applicable)
3. Perform the calculation
4. Explain the result

Question: {question}

Solution:""",
)

advanced_cot_chain = advanced_cot_prompt | llm

complex_question = "A car travels 150 km at 60 km/h, then another 100 km at 50 km/h. What is the average speed for the entire journey?"

advanced_cot_response = advanced_cot_chain.invoke(complex_question).content
print(advanced_cot_response)

To solve the problem, we need to find the average speed for the entire journey. Here are the steps to calculate it:

### Step 1: Calculate the total distance traveled
1. **What we're going to calculate:** The total distance traveled.
2. **Formula:** None needed, just addition.
3. **Calculation:** 
   \[
   \text{Total distance} = 150 \, \text{km} + 100 \, \text{km} = 250 \, \text{km}
   \]
4. **Explanation:** The total distance is the sum of the two segments of the journey, which is 250 km.

### Step 2: Calculate the time taken for each segment of the journey
1. **What we're going to calculate:** The time taken for each segment of the journey.
2. **Formula:** \( \text{Time} = \frac{\text{Distance}}{\text{Speed}} \)
3. **Calculation for the first segment:**
   \[
   \text{Time}_1 = \frac{150 \, \text{km}}{60 \, \text{km/h}} = 2.5 \, \text{hours}
   \]
4. **Calculation for the second segment:**
   \[
   \text{Time}_2 = \frac{100 \, \text{km}}{50 \, \text{km/h}} = 2 \, \text{hours}
   \]


## Auto Chain of Thought Techniques

Now, let's explore a technique that automatically generates CoT prompts.

In [5]:
reasoning_llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="amazon/nova-pro-v1",
)

auto_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
Generate a general chain of thought prompt to solve the following question.
Please do not attempt to solve the question yourself.
Instead, generate a prompt that will help a language model to solve the question step by step.

Question: {question}

Chain of thought prompt:""",
)

auto_cot_chain = auto_cot_prompt | reasoning_llm

complex_question = "A car travels 150 km at 60 km/h, then another 100 km at 50 km/h. What is the average speed for the entire journey?"

auto_cot_response = auto_cot_chain.invoke(complex_question).content
print(auto_cot_response)

To determine the average speed for the entire journey, we need to follow these steps:

1. **Calculate the time taken for each segment of the journey:**
   - For the first segment: 
     - Distance = 150 km
     - Speed = 60 km/h
     - Use the formula: time = distance / speed
   - For the second segment:
     - Distance = 100 km
     - Speed = 50 km/h
     - Again, use the formula: time = distance / speed

2. **Sum the times for both segments to get the total time taken for the entire journey.**

3. **Calculate the total distance traveled:**
   - Add the distances of both segments: 150 km + 100 km

4. **Determine the average speed:**
   - Use the formula for average speed: average speed = total distance / total time

By following these steps, you can find the average speed for the entire journey.


In [6]:
# 1. Keep existing auto_cot_chain
auto_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""Generate a general chain of thought prompt to solve the following question.
Please do not attempt to solve the question yourself.
Instead, generate a prompt that will help a language model to solve the question step by step.
Question: {question}
Chain of thought prompt:""",
)
auto_cot_chain = auto_cot_prompt | reasoning_llm

# 2. Modify advanced prompt to include generated CoT
advanced_cot_prompt = PromptTemplate(
    input_variables=["generated_cot", "question"],
    template="""
{generated_cot}
Question: {question}
Solution:""",
)

# 3. Create chained implementation
advanced_cot_chain = (
    RunnablePassthrough.assign(generated_cot=auto_cot_chain) | advanced_cot_prompt | llm
)

# 4. Invoke together
complex_question = "A car travels 150 km at 60 km/h, then another 100 km at 50 km/h. What is the average speed for the entire journey?"
response = advanced_cot_chain.invoke({"question": complex_question})
print(response.content)

To determine the average speed for the entire journey, we need to follow these steps:

1. **Calculate the time taken for each segment of the journey:**
   - For the first segment:
     - Distance = 150 km
     - Speed = 60 km/h
     - Use the formula: time = distance / speed
     - Time for the first segment = 150 km / 60 km/h = 2.5 hours

   - For the second segment:
     - Distance = 100 km
     - Speed = 50 km/h
     - Use the formula: time = distance / speed
     - Time for the second segment = 100 km / 50 km/h = 2 hours

2. **Sum the times calculated for both segments to get the total time for the journey:**
   - Total time = Time for the first segment + Time for the second segment
   - Total time = 2.5 hours + 2 hours = 4.5 hours

3. **Calculate the total distance traveled:**
   - Add the distances of both segments: 150 km + 100 km = 250 km

4. **Determine the average speed using the formula:**
   - Average speed = total distance / total time
   - Average speed = 250 km / 4.5 hou

## Comparative Analysis

Let's compare the effectiveness of standard prompting vs. CoT prompting on a more challenging problem.

In [7]:
challenging_question = """
A cylindrical water tank with a radius of 1.5 meters and a height of 4 meters is 2/3 full. 
If water is being added at a rate of 10 liters per minute, how long will it take for the tank to overflow? 
Give your answer in hours and minutes, rounded to the nearest minute. 
(Use 3.14159 for π and 1000 liters = 1 cubic meter)"""

standard_response = standard_chain.invoke(challenging_question).content
cot_response = advanced_cot_chain.invoke({"question": challenging_question}).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)

Standard Response:
11 hours and 48 minutes

Chain of Thought Response:
To solve this problem, follow these steps:

1. **Determine the volume of the cylindrical tank:**
   - Recall the formula for the volume of a cylinder: \( V = \pi r^2 h \).
   - Substitute the given radius \( r = 1.5 \) meters and height \( h = 4 \) meters into the formula.
   - Calculate the volume of the tank in cubic meters.
   
   \[
   V = \pi \times (1.5)^2 \times 4 = 3.14159 \times 2.25 \times 4 = 28.2743 \text{ cubic meters}
   \]

2. **Calculate the current volume of water in the tank:**
   - The tank is 2/3 full. Multiply the total volume of the tank by \( \frac{2}{3} \) to find the current volume of water.
   
   \[
   \text{Current volume} = \frac{2}{3} \times 28.2743 = 18.8495 \text{ cubic meters}
   \]

3. **Determine the remaining volume needed to fill the tank:**
   - Subtract the current volume of water from the total volume of the tank to find the remaining volume that needs to be filled.
   
   \[
